# Perform regression on parameterized surfaces

## Imports

In [1]:
# imports

import trimesh
import os
import sys
import subprocess
import torch
import numpy as np
os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs
import my28brains.discrete_surfaces

INFO: Using pytorch backend


In [2]:
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()

my28brains_dir = os.path.join(work_dir, "my28brains")

sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
sys.path.append(my28brains_dir)

# this forces the notebook to re-load
import importlib
importlib.reload(my28brains.discrete_surfaces)

from my28brains.discrete_surfaces import (
    DiscreteSurfaces,
    ElasticMetric,
)

import my28brains.default_config as default_config

## Generate parameterized geodesic from sphere to ellipse using `discrete_curves.py`.

In [3]:
#create sphere mesh
sphere = trimesh.creation.icosphere(subdivisions=3, radius=30.0)

#Create ellipsoid mesh
# Create a scaling matrix for the semi-axes lengths
scales = np.array([2, 2, 3])
scale_matrix = np.diag(scales)

scale_matrix = gs.array(scale_matrix)

# Apply the scaling transformation to the mesh vertices
scaled_vertices = sphere.vertices.dot(scale_matrix)

# Create a new mesh with the scaled vertices
ellipsoid = trimesh.Trimesh(vertices=scaled_vertices, faces=sphere.faces)

#Define Surface Space
SURFACE_SPACE = DiscreteSurfaces(faces = sphere.faces)

METRIC = ElasticMetric(
    space = SURFACE_SPACE,
    a0=default_config.a0,
    a1=default_config.a1,
    b1=default_config.b1,
    c1=default_config.c1,
    d1=default_config.d1,
    a2=default_config.a2,)

# CREATE GEODESIC
n_vertices = sphere.vertices.shape[0]
print(n_vertices)


def geodesics_sphere_to_ellipsoid(
    n_geodesics=1, n_times=5
):
    """Generate a dataset of geodesics that transform spheres into ellipsoids."""
    dim = 3
    
    geodesics = gs.zeros((n_geodesics, n_times, n_vertices, dim))
    times = gs.arange(0, 1, 1 / n_times)
    for i_geodesic in range(n_geodesics):
        start_surface = gs.array(sphere.vertices)
        end_surface = gs.array(ellipsoid.vertices)
        print(start_surface.shape)
        geodesic = METRIC.geodesic(initial_point = start_surface, end_point = end_surface)
        
        geodesics[i_geodesic] = geodesic(times)

    return geodesics, times

geodesics, times = geodesics_sphere_to_ellipsoid()
geodesic = geodesics[0]
print(geodesic.shape)

642
torch.Size([642, 3])
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5778     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.86728D+08    |proj g|=  4.99244D+04


 This problem is unconstrained.



At iterate    1    f=  3.15951D+08    |proj g|=  1.37249D+05

At iterate    2    f=  2.50530D+08    |proj g|=  4.32341D+05

At iterate    3    f=  2.20899D+08    |proj g|=  5.06246D+05

At iterate    4    f=  1.84697D+08    |proj g|=  5.33675D+04

At iterate    5    f=  1.81563D+08    |proj g|=  3.93044D+04

At iterate    6    f=  1.74483D+08    |proj g|=  9.26942D+04

At iterate    7    f=  1.73475D+08    |proj g|=  1.84078D+05

At iterate    8    f=  1.68764D+08    |proj g|=  6.48147D+04

At iterate    9    f=  1.66201D+08    |proj g|=  2.71185D+04

At iterate   10    f=  1.62078D+08    |proj g|=  6.22908D+04

At iterate   11    f=  1.56385D+08    |proj g|=  8.33474D+04

At iterate   12    f=  1.46889D+08    |proj g|=  2.01747D+05

At iterate   13    f=  1.39824D+08    |proj g|=  1.59422D+05

At iterate   14    f=  1.32977D+08    |proj g|=  4.42927D+04

At iterate   15    f=  1.27929D+08    |proj g|=  4.36129D+04

At iterate   16    f=  1.23887D+08    |proj g|=  4.47398D+04

At iter

# GEODESIC REGRESSION

In [4]:
# Regression imports

import matplotlib.pyplot as plt

import geomstats.backend as gs
import geomstats.visualization as visualization
from geomstats.geometry.special_euclidean import SpecialEuclidean
from geomstats.learning.frechet_mean import FrechetMean, variance
from geomstats.learning.geodesic_regression import GeodesicRegression

In [ ]:
gr = GeodesicRegression(
        SURFACE_SPACE,
        metric=METRIC,
        center_X=False,
        method="riemannian",
        max_iter=100,
        init_step_size=0.1,
        verbose=True,
        initialization="frechet",
    )

# X = gs.random.normal(size=(5))
# X -= gs.mean(X)
# print(geodesic.shape)

gr.fit(times, geodesic, compute_training_score=True)

intercept_hat, beta_hat = gr.intercept_, gr.coef_

 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5778     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  0.00000D+00    |proj g|=  0.00000D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5778      0      1      0     0     0   0.000D+00   0.000D+00
  F =   0.0000000000000000     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5778     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.4

 This problem is unconstrained.



At iterate    1    f=  1.86622D+07    |proj g|=  3.80617D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5778      1      3      1     0     0   3.806D+02   1.866D+07
  F =   18662172.117284276     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5778     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.69182D+07    |proj g|=  7.63575D+02


 This problem is unconstrained.



At iterate    1    f=  4.69155D+07    |proj g|=  1.41438D+03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5778      1      3      1     0     0   1.414D+03   4.692D+07
  F =   46915490.833080284     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5778     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.36869D+07    |proj g|=  2.31308D+03

At iterate    1    f=  9.36708D+07    |proj g|=  3.97712D+03

           * * *

Tit   = total number of iterations
Tnf   = total nu

 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         1926     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.55706D+02    |proj g|=  3.56421D+04

At iterate    1    f=  5.50778D+02    |proj g|=  1.42953D+04

At iterate    2    f=  5.11827D+02    |proj g|=  7.99771D+03

At iterate    3    f=  4.65380D+02    |proj g|=  4.99238D+03

At iterate    4    f=  4.33992D+02    |proj g|=  7.32999D+03

At iterate    5    f=  4.12889D+02    |proj g|=  1.09676D+04

At iterate    6    f=  3.83662D+02    |proj g|=  5.11089D+03

At iterate    7    f=  3.53524D+02    |proj g|=  5.99643D+03

At iterate    8    f=  3.23046D+02    |proj g|=  5.65681D+03

At iterate    9    f=  2.95224D+02    |proj g|=  1.04609D+04

At iterate   10    f=  2.73470D+02    |proj g|=  5.54403D+03

At iterate   11    f=  2.62842D+02    |proj g|=  3.56569D+03

At iterate   12    f=  2.47451D+02    |proj g|=  3.95446D+03

At iterate   13    f=  2.3